## Imports 

In [1]:
#Versión para tweepy V2
from datetime import date
import datetime
import requests
import json
import time
from pathlib import Path
import math
import pymongo
from bson import ObjectId
import numpy as np
from datetime import date


## Constant and Variables

In [2]:
# CONSTANTS

PETITIONS_PER_TIME_LAPSE = 300
TIME_LAPSE = 20*60
# Tiempo de espera necesario entre peticiones
TIMER = TIME_LAPSE/PETITIONS_PER_TIME_LAPSE
# Log cada 2 minutos
TIMER_LOG = 2*60

#porcentaje de tweets que nos quedamos para cada dia
PERCENT = 0.15
# máximo de tweets por cada petición
PETITIONS_LIMIT = 500

#VARIABLES
iniTimerLog = 0
finTimerLog = 0
tiempo = TIMER_LOG
hashtagsList = []
hashtags = []
entrada = ""
next_token = None
entradaizq = '' 
entradader = ''
total = 0
startDate = ''
endDate = ''
inicio = 0
fin = 0

hashtag_ini_log = ''
start_log_timer = ''
hashtag_last_log = ''
tweet_count = 0

url = 'https://api.twitter.com/2/tweets/search/all?'
today = datetime.date.today()
d1 = today.strftime("%d-%m-%Y")



## Support functions

In [3]:
user_dictionary = {
    "_id"               : None,
    "description"       : '',
    "created_at"        : '',
    "verified"          : '',
    "followers_count"   : 0,
    "following_count"   : 0
}


tweet_dictionary = {
    "_id"               : None,
    "author_id"         : '',
    "created_at"        : '',
    "text"              : '',
    "hashtag"           : [],
    "referenced_tweets" : [],
    "retweet_count"     : 0,
    "reply_count"       : 0,
    "reply_count"       : 0,
    "like_count"        : 0
}


node = {
    "hashtag"           : '',
    "current_date"        : '',
    "next_date"        : '',
    "next_token"        : '-1'
}


def DayMonthYearToPetition(date):
    """
    """
    splited = ''
    if ('/' in date):
        splited = date.split('/')
    elif ('-' in date):
        splited = date.split('-')

    return (splited[2] + '-' + splited[1] + '-' + splited[0])

def DayMonthYearToPetition(date):
    """
    """
    splited = ''
    if ('/' in date):
        splited = date.split('/')
    elif ('-' in date):
        splited = date.split('-')

    return (splited[2] + '-' + splited[1] + '-' + splited[0])

def PetitionToDayMonthYear(petition):
    """
    """
    splited = ''
    time = ''
    splited = petition.split('T')
    time = splited[1]
    splited = splited[0].split('-')
    return (splited[2] + '-' + splited[1] + '-' + splited[0] + 'T' + time)

def make_objid(text):
    """Makes an ObjectId of 4 bytes
    
    Keyword arguments:
    text -- string to be converted into Object ID
    """
    text = str(text)
    if not text.strip():
        return None
    try:
        return ObjectId(text.rjust(24,"0"))
    except Exception as ex:
        #print(len(text))
        print(text, ex)
        return None

def create_headers(bearer_token):
    """
    """
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def connect_to_endpoint(headers, params):
    """
    """
    response = requests.request("GET", "https://api.twitter.com/2/tweets/search/all?", headers=headers, params=params, timeout=2)
    #print(response.status_code)
    # Manejo de errores
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def MongoExecute(tweet_list, user_list):
    """
    """
    global myclient
    global myusers
    global mytweets
    global mydb
    try:
        print(str(mytweets.insert_many(tweet_list, ordered=False)))
        print(str(myusers.insert_many(user_list, ordered=False)))
    except Exception as e:        
        pass

def PrintTweetsUsers(tweets, users):
    print("Tweet list")
    for tweet in tweets:
        print(" id: " + str(tweet['_id']))
        print(" author_id: " + tweet['author_id'])
        print(" created_at: " + tweet['created_at'])
        print(" referenced_tweets: " + str(tweet['referenced_tweets']) + '\n')

    print("User list")
    for user in users:
        print(" user_id: " + str(user['_id']))
        print(" description: " + user['description'])
        print(" created_at: " + user['created_at'])
        print(" verified: " + str(user['verified']))
        print(" followers_count: " + str(user['followers_count']))
        print(" following_count: " + str(user['following_count']) + '\n')
"""
     Insertados los siguientes campos:
        -start_time: límite inferior en búsqueda de fechas
        -end_time: límete superior en búsqueda de fechas
        -tweet.fields=author_id: id del autor del tweet
        -tweet.fields=referenced_tweets: puede ser quote, retweet o replied_to junto al id al que responde, agrega además los tweets relativos al tweet al que responde
        -tweet.fields=created_at: cu'ando se cre'o el fichero
        -tweet.fields=context_annotations:anotaciones de contexto (no salen muchas)
        -tweet.fields=lang: Abreviatura del idioma en el que se escribe
        -tweet.fields=entities: referencias, hashtags usados por el usuario
        -tweet.fields=in_reply_to_user_id: 
        -in_reply_to_user_id

"""

# Hashtag: nombre del hashtag que se va a buscar
# start: fecha de inicio
# end: fecha de respuesta
# number: n'umero de tweets que queremos obtener
# bearer: bearer id
# next_token: token de la page (inicialmente string vac'ia) que quieres buscar
# return tupla de n'umero de respuestas y next_token
def PetitionsLessEqualPETITIONS_LIMIT(hashtag, start, end, bearer, next_token):

    global log
    global errorlog
    global headers
    global PETITIONS_LIMIT
    global hashtagsList
    global hashtags

    startDate = DayMonthYearToPetition(start)
    endDate = DayMonthYearToPetition(end)
    tweet_list = []
    user_list = []
    total = (0, '', [], [])
    dt_string = ''
    tweet_dictionary_new = {}
    user_dictionary_new = {}
    aux = ''
    

    params = "query=%23" + hashtag + "&start_time=" +  startDate + "T00%3A00%3A00Z" + "&end_time=" + endDate + 'T11%3A59%3A59Z&max_results=' + str(PETITIONS_LIMIT) + "&expansions=author_id,referenced_tweets.id&tweet.fields=created_at,conversation_id,referenced_tweets,public_metrics,text&user.fields=created_at,entities,description,verified,public_metrics"

    # caso en el que -1 (primera peticion de dia) o es el next_token
    #print("next_token ")
    if (next_token != ''):
        params = params + '&next_token=' + next_token

    json_obj = ''
    #print("try de connect_to_endpoint ")
    try:
        json_obj = connect_to_endpoint(headers=headers, params=params)
    except Exception as e:
        print(e)
        return (0, next_token, [], [])
    
    #print("if meta ")
    # Comprobar que hay respuestas para dicho hashtag
    if ('meta' in json_obj) and (int(json_obj['meta']['result_count']) > 0):
        #print("respuesta es meta")
        for tweet in json_obj['data']:
            #print(" creacion de tweet_dictionary_new")
            tweet_dictionary_new = tweet_dictionary.copy()
            tweet_dictionary_new['hashtag'] = []
            tweet_dictionary_new['referenced_tweets'] = []
            user_dictionary_new = user_dictionary.copy()
            
            tweet_dictionary_new['_id'] = make_objid(str(tweet['id']))
            tweet_dictionary_new['author_id'] = make_objid(str(tweet['author_id']))
            tweet_dictionary_new['hashtag'].append(hashtag.lower())

            tweet_dictionary_new['created_at'] = tweet['created_at']
            tweet_dictionary_new['retweet_count'] = tweet['public_metrics']['retweet_count']
            tweet_dictionary_new['reply_count'] = tweet['public_metrics']['reply_count']
            tweet_dictionary_new['like_count'] = tweet['public_metrics']['like_count']
            tweet_dictionary_new['quote_count'] = tweet['public_metrics']['quote_count']
            
            if ('referenced_tweets' in tweet):
                #print("  creacion de referenced_tweets")
                tweet_dictionary_new['referenced_tweets'] = tweet['referenced_tweets']
                for rt in tweet_dictionary_new['referenced_tweets']:
                    for tweetexpanse in json_obj['includes']['tweets']:
                        if (rt['id'] == tweetexpanse['id']):
                            tweet_dictionary_new['text'] = tweetexpanse['text']
                            aux = ''
                            #esto se puede hacer una funci'on a parte
                            for word in tweet['text'].split():
                                aux = word.rstrip('\n')
                                aux = word.replace('#', '')
                                for h in hashtags:
                                    if aux.lower() == h.lower().rstrip('\n').replace('#', '') and aux.lower () != hashtag.lower():
                                        tweet_dictionary_new['hashtag'].append(hashtag.lower())
                            rt['author_id'] = make_objid(str(tweetexpanse['author_id']))
                            rt['id'] = make_objid(str(tweetexpanse['id']))
                            break
                #print("   fin de referenced_tweets")
            else:
                #print("  creacion de text")
                tweet_dictionary_new['text'] = tweet['text']
                #print("   fin de text")

            unknown = True
            #print("    inicio de unknown")
            for user in user_list:
                if (str(user['_id']) == str(tweet_dictionary_new['author_id'])):
                    unknown = False
                    break
            #print("     fin de unknown")

            if (unknown == True):
                #print("      inicio true de unknown")
                user_dictionary_new['_id'] = make_objid(str(tweet['author_id']))
                if ('includes' in json_obj):
                    for user in json_obj['includes']['users']:
                        if (str(make_objid(user['id'])) == str(user_dictionary_new['_id'])):
                            user_dictionary_new['description'] = user['description']
                            user_dictionary_new['created_at'] = user['created_at']
                            user_dictionary_new['verified'] = user['verified']
                            user_dictionary_new['followers_count'] = int(user['public_metrics']['followers_count'])
                            user_dictionary_new['following_count'] = int(user['public_metrics']['following_count'])
                            break
                
                #aqui otro manejo de errores
                else:
                    print ('El formato del mensaje recibido no es el correcto', str(json_obj))
                #print("       fin true de unknown")
                #print("        inicio inserci'on user_dictionary_new en lista")
                user_list.append(user_dictionary_new)
            #print("        inicio inserci'on tweet_dictionary_new en lista")
            tweet_list.append(tweet_dictionary_new)

        if ('next_token' in json_obj['meta']):
            #print("         next_token")
            total = (int(json_obj['meta']['result_count']), json_obj['meta']['next_token'], tweet_list, user_list)
        else:
            #print("         no hay next_token")
            total = (int(json_obj['meta']['result_count']), '', tweet_list, user_list)

        print("     N'umero de peticiones enviadas: " + str(PETITIONS_LIMIT))
        print("     N'umero de peticiones recibidas: " + str(json_obj['meta']['result_count']))
        print("     " + str(total[1]))
        return total
    # Caso en el que la respuesta es un error y hay que manejarlo
    else:
        #print("else error ")
        strerror = '    '
        #print("Error en la petición: ")
        strerror = strerror + str(params)
        #print(entrada.split(' -H')[0])
        strerror = strerror + "Error en la petición: "
        # si el error es por cuestión de peticiones esperar para hacer la siguiente peticion
        if ('title' in json_obj):
            #print("  Título del error: " + str(json_obj['title']))
            strerror = strerror + "  Título del error: " + str(json_obj['title'])
            if ('status' in json_obj):
                strerror = strerror + " Código de error: " + str(json_obj['status'])
                #print("  Código de error: " + str(json_obj['status']))
            if ('detail' in json_obj):
                strerror = strerror + "  Descripción del error: " + str(json_obj['detail'])
                #print("  Descripción del error: " + str(json_obj['detail']))
                
        #errorlog.write(strerror + " " + dt_string + '\n')
        total = (0, next_token, [], [])
    return total




# Hashtag: nombre del hashtag que se va a buscar
# start: fecha de inicio
# end: fecha de respuesta
# number: n'umero de tweets que queremos obtener
# bearer: bearer id
# return devuelve dos diccionarios (users-tweet, tweet-tweet_items)
def TweetList(hashtag, start, end, bearer, token):
    global tiempo
    global TIMER
    global TIMER_LOG
    global tweet_count
    global hashtag_ini_log
    global hashtag_last_log
    global start_log_timer
    

    #aqui deberia de hacer dos variables de incio y fin de log para saber con cual hashtag empiezo y termino y con cuantos valores totales de cada uno.QUITAR TWEETLIST
    total = 0
    aux = 0
    startTime = 0
    endTime = 0
    response = (0, token, [], [])

    startTime = time.time()

    #print("Inicio peticioneslessEqual")
    response =  PetitionsLessEqualPETITIONS_LIMIT(hashtag, start, end, bearer, response[1])
    #print("Fin peticioneslessEqual ", str((str(response[0]), str(response[1]), str(len(response[2])), str(len(response[3])))))

    #inserci'on de tweets nuevos a la lista de tweets
    total = response[0]
    tweet_count += response[0]
    endTime = time.time()

    aux = round(math.floor(endTime - startTime))
    if (aux < TIMER):
        number = TIMER - aux
        time.sleep(number)
        #print('     sleep number: ' + str(number) + ', aux: ' + str(aux), flush=True)
        tiempo -= TIMER
    else:
        tiempo -= aux
        #print('     aux: ' + str(aux), flush=True)
    
    if (tiempo <= 0):
        #aqui se imprime en el log los hashtags que se han escrito y c'uantos valores se han obtenido de estos
        tiempo = TIMER_LOG
        # datetime object containing current date and time
        #now = datetime.datetime.now()
        # dd/mm/YY H:M:S
        #dt_string = now.strftime("%H:%M:%S")
        #hashtag_last_log = hashtag
        #log.write(hashtag_ini_log + ' ' + start_log_timer + ' ' + hashtag + ' ' + start + ' ' + dt_string + '\n')
        #start_log_timer = dt_string
        #tweet_count = 0
        #hashtag_ini_log = hashtag_last_log
    
    print ("    Tweets totales: " + str(total) + '\n')

    return (total, response[1], response[2], response[3])


def GetTweets(start_date, end_date):


    global tiempo
    global TIMER
    global TIMER_LOG
    global tweet_count
    global hashtags

    aux = end_date.split('-')
    last_date = date(int(aux[2]), int(aux[1]), int(aux[0]))
    last_date = last_date.strftime("%d/%m/%Y")

    f =  open("./hashtags.txt", "r")
    hashtags = f.readlines()
    f.close()
    
    if os.path.exists("./lasttoken.txt") and os.path.getsize("./lasttoken.txt") > 0:
        f =  open("./lasttoken.txt", "r")
        while True:
            # read line
            try:
                hashtag = f.readline()
                if (hashtag == "") :
                    break
            except:
                # datetime object containing current date and time
                now = datetime.datetime.now()
                # dd/mm/YY H:M:S
                dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
                errorlog.write("Fallo al leer el fichero hashtags.txt " + dt_string + "\n")

            hashtag = hashtag.rstrip('\n')
            hashtag = hashtag.split(' ')
            node_aux = node.copy()
            node_aux['hashtag'] = hashtag[0]
            node_aux['next_token'] = hashtag[1]
            node_aux['current_date'] = hashtag[2]
            node_aux["next_date"] = hashtag[3]  
            hashtagsList.append(node_aux)
        f.close()

        fd=open("./lasttoken.txt","r")
        d=fd.read()
        fd.close()
        m=d.split("\n")
        s="\n".join(m[:-1])

        fd=open("./lasttoken.txt","w+")
        for i in range(len(s)):
            fd.write(s[i])
        fd.close()
        
    else:
        #####Uso de Hashtag
        f =  open("./hashtags.txt", "r")
        d =  open("./lasttoken.txt", "w+")
        
        writeable = ''
        hashtag = None
        resultado = (0, '', [], [])
        aux = start_date.split('-')
        dat = date(int(aux[2]), int(aux[1]), int(aux[0]))
        first_day = dat
        dat += datetime.timedelta(days=1)
        next_day = dat
        str_first_day = first_day.strftime("%d/%m/%Y")
        str_next_day = next_day.strftime("%d/%m/%Y")
        while True:
            # read line
            try:
                hashtag = f.readline()
                if (hashtag == "") :
                    break
            except:
                # datetime object containing current date and time
                now = datetime.datetime.now()
                # dd/mm/YY H:M:S
                dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
                errorlog.write("    Fallo al leer el fichero hashtags.txt " + dt_string + "\n")
            
            hashtag = hashtag.rstrip('\n')
            hashtag = hashtag.replace('#', '')
            
            node_aux = node.copy()
            node_aux["hashtag"] = hashtag
            writeable = hashtag

            writeable += ' ' + node_aux['next_token']

            node_aux["current_date"] = str_first_day   
            writeable += ' ' + str_first_day 

            node_aux["next_date"] = str_next_day 
            writeable += ' ' + str_next_day + '\n'   

            hashtagsList.append(node_aux)
            d.write(writeable)
            # check if line is not empty
        d.close()
    f.close()

        

    for h in hashtagsList:
        print(str(h))
        


    if (len(hashtagsList) != 0):
        hashtag_ini_log = hashtagsList[0]
        # dd/mm/YY H:M:S
        now = datetime.datetime.now()
        dt_string = now.strftime("%H:%M:%S")
        start_log_timer = dt_string
        
    else:
        # datetime object containing current date and time
        now = datetime.datetime.now()
        # dd/mm/YY H:M:S
        dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
        errorlog.write("    Error: lista de hashtags vacía" + " dt_string\n")

    count = 0
    temporalTokens = ''
    tweet_list = []
    user_list = []
    start = 0
    end = 0
    
    last = False
    while last == False:
        for l in hashtagsList:
            start = 0
            end = 0
            aux = 0
            #bucle de obtencion de tweets para un dia concreto
            while (l["next_token"] != ''):
                #print('Inicio while, ', str((resultado[0], resultado[1], len(resultado[2]), len(resultado[3]))))
                if l["next_token"] == '-1':
                    print('Inicio hashtag: ' + l['hashtag'] + ' día: ' + l['current_date'])
                    resultado = TweetList(l["hashtag"], l["current_date"], l["next_date"], Bearer, '')
                else:
                    resultado = TweetList(l["hashtag"], l["current_date"], l["next_date"], Bearer, l["next_token"])
                    #print('Inicio resultado, ', str((resultado[0], resultado[1], len(resultado[2]), len(resultado[3]))))
                start = time.time()
                tweet_list.extend(resultado[2])
                #print('llega aqui resultado 2 ', len(resultado[2]))
                user_list.extend(resultado[3])
                #print('llega aqui resultado 3 ', len(resultado[3]))
                l["next_token"] = resultado[1]
                #print('llega aqui next_token ', l["next_token"])
                end = time.time()

                tiempo -= round(math.floor(start - end))
                #print('llega aqui tiempo ', str(tiempo))

                if (tiempo <= 0):
                    # datetime object containing current date and time
                    tiempo = TIMER_LOG
                    #now = datetime.datetime.now()
                    # dd/mm/YY H:M:S
                    #dt_string = now.strftime("%H:%M:%S")
                    #log.write(hashtag_ini_log + ' ' + start_log_timer + ' ' + l['hashtag'] + ' ' + l['current_date'] + ' ' + dt_string + '\n')
                    #hashtag_ini_log = l['hashtag']

            start = time.time()
            # Substring de % para insertar en la base de datos
            count = 0
            count = round(math.ceil(len(tweet_list) * PERCENT))
            if count > 0:
                tweet_list = tweet_list[0:(count - 1)]
            #caso en el que el redondeado de un número entre 0 y 1
            elif count == 0 and len(tweet_list) > 0:
                tweet_list = tweet_list[0]

            aux_list = []
            # Ahora buscamos los users relacionados con los tweets de la lista
            for u in user_list:
                for t in tweet_list:
                    #decisi'on de dise;o, como habr'a mismo usuario repetido para distintos tweets el insert no insertar'a el mismo usuario
                    #y al estar insertado en paralelo estar'a insertado de forma eficiente
                    if (str(t['author_id']) == str(u['_id'])):
                        aux_list.append(u)
                        break 
            print("Total tweets insertados: " + str(len(tweet_list)) + ", Total usuarios insertados: " + str(len(aux_list)))

            MongoExecute(tweet_list, aux_list)

            
            #ahora cambiamos de d'ia y quitamos la lista
            tweet_list = []
            user_list = []
            print('     Cambio de día ' + l['current_date'] + '\n')
            l['current_date'] = l['next_date']
            dat = datetime.datetime.strptime(l['current_date'], "%d/%m/%Y")
            dat += datetime.timedelta(days=1)
            l['next_date'] = dat.strftime("%d/%m/%Y").split(' ')[0]
            l['next_token'] = '-1'
            #borramos la 'ultima linea de fichero de lasttoken
            if os.path.exists("./lasttoken.txt") and os.path.getsize("./lasttoken.txt") > 0:
                d = ''
                f =  open("./lasttoken.txt", "r")
                #remove last line from a text line in python
                d = f.readlines()
                f.close()
                
                for i in range(len(d)):
                    if d[i].split()[0] == l['hashtag']:
                        d[i] = l['hashtag'] + ' ' + l['next_token'] + ' ' + l['current_date'] + ' ' + l["next_date"] + '\n'

                f = open("./lasttoken.txt","w+")
                f.writelines(d)
                f.close()

            if (l["current_date"] == last_date):
                last = True

            end = time.time()
                
            tiempo -= round(math.floor(start - end))
            if (tiempo <= 0):
                # datetime object containing current date and time
                tiempo = TIMER_LOG
                #now = datetime.datetime.now()
                # dd/mm/YY H:M:S
                #dt_string = now.strftime("%H:%M:%S")
                #log.write(hashtag_ini_log + ' ' + start_log_timer + ' ' + l['hashtag'] + ' ' + l['current_date'] + ' ' + dt_string + '\n')
                #hashtag_ini_log = l['hashtag']

            print("Fin Hashtag: " + str(l['hashtag']) + "\n\n")        

    log.close()
    errorlog.close()

    
    


## Main code

In [ ]:
# Si ya se ha buscado anteriormente entonces deberías de comprobar si la búsqueda y lo que se pide de base (n_teets no se que es igual al número de tweets del hashtag)

myclient = pymongo.MongoClient('localhost', 27017)
mydb = myclient["mydatabase"]
myusers = mydb["users"]
mytweets = mydb["tweets"]

f = open("./accountV2.txt", "r")
Bearer = f.readline().rstrip('\n')
f.close()
headers = create_headers(Bearer)

fileObj = Path("./log" + d1 + ".txt")
if (fileObj.is_file()):
    os.remove("./log" + d1 + ".txt")

fileObj = Path("./errorlog" + d1 + ".txt")
if (fileObj.is_file()):
    os.remove("./errorlog" + d1 + ".txt")

log = open("./log" + d1 + ".txt", "a")
log.write("Primer log\n")
#Crear ficheros de log
errorlog = open("./errorlog" + d1 + ".txt", "a")


GetTweets("19-03-2021", "04-05-2021")
